
# (some) Related Work

# abstract
We write this when p5 is done.

# Boundary Value Problems
In period2 we solved a boundary problem using Green functions. Our goal for the next period is to get an implementation of unbiased $1$ D BVP solver of higher order. In the following we implement different BVP  algorithms for
$$
y''=y, y(-1) = e^{-1},y(1)=e .
$$

## Preparation
For those algorithms we need the corresponding Green functions of this problem.

### boundary green function of $y''$

$$
\begin{align*}
G''_{b_{0}}(t) &= 0, G(b_{0})=1,G(b_{1})=0 \Rightarrow \\
G_{b_{0}}(t) &= \frac{b_{1}-t}{b_{1}-b_{0}} \\   
\end{align*}
$$ 
$$
\begin{align*}
G''_{b_{1}}(t) &= 0, G(b_{0})=0,G(b_{1})=1 \Rightarrow \\
G_{b1}(t) &= \frac{t-b_{0}}{b_{1} -b_{0}}
\end{align*}
$$


### source green function of $y''$
$$
\begin{align*}
G''_{s}(t) &= \delta(t-s), G(b_{0})=0,G(b_{1})=0 \Rightarrow \\
G_{s}(t) &=    \begin{cases}
        -\frac{(b_{1}-s)(t-b_{0})}{b_{1}-b_{0}} & \text{if } t<s\\
        -\frac{(b_{1}-t)(s-b_{0})}{b_{1}-b_{0}} & \text{if } s<t
    \end{cases} \\
G_{t}(s) &=    \begin{cases}
        -\frac{(b_{1}-t)(s-b_{0})}{b_{1}-b_{0}} & \text{if } s<t\\
        -\frac{(b_{1}-s)(t-b_{0})}{b_{1}-b_{0}} & \text{if } t<s
    \end{cases}
\end{align*} 
$$ 

### testing the green function
We solve
$$
y''= y,y(b_{0}), y(b_{1}).
$$ 
the same way as in period2 to test these green functions are correct

In [19]:
from random import random
from math import exp
import numpy as np

def Gb0(t,b0,b1): return (b1-t)/(b1-b0)
def Gb1(t,b0,b1): return (t-b0)/(b1-b0)
def G(t,s,b0,b1): return -(b1-s)*(t-b0)/(b1-b0) if t<s else - (b1-t)*(s-b0)/(b1-b0) 

def Y(t,y0,y1,b0,b1): 
    l = 2 # russian roulette rate
    if random()*l>1: return Gb0(t,b0,b1)*y0 + Gb1(t,b0,b1)*y1
    S = b0+random()*(b1-b0)
    return Gb0(t,b0,b1)*y0 + Gb1(t,b0,b1)*y1 + l*G(t,S,b0,b1) * Y(S,y0,y1,b0,b1)*(b1-b0)
    
def soltest(t,y0,y1,b0,b1,nsim): return sum(Y(t,y0,y1,b0,b1) for _ in range(nsim))/nsim

nsim = 10**2
B0 = np.arange(-1,0,0.2)
B1 = np.arange(1,0,-0.2)
T = [b0+(b1-b0)*random() for b0,b1 in zip(B0,B1) ]

for t,b0,b1 in zip(T,B0,B1):
    print((soltest(t,exp(b0),exp(b1),b0,b1,nsim)-exp(t))/exp(t))


print(20*"-")
nsim = 10**2
B0 = list(np.arange(-5,-1,1)) + [-0.5]
B1 = list(np.arange(5,1,-1)) + [0.5]
T = [0]*len(B0)

for t,b0,b1 in zip(T,B0,B1):
    print((soltest(t,exp(b0),exp(b1),b0,b1,nsim)-exp(t))/exp(t))

0.02216861549131576
-0.0208313414490792
-0.026617346220855084
0.007517183795474667
-0.003799054516548625
--------------------
-131755476.57755683
314902.9346051018
4968.071633233332
8456.558236166946


## WoS (Walk on Sphere)

The idea of walk on sphere is to use the integral equation corresponding to the largest sphere around $t$ contained in the domain:
![WoS](tests_period_5/tikz%20figures/WoS/WoS.png)
$$
y(t) = G_{h_{0}}(t)y_{h_{0}} + G_{h_{1}}(t) y_{h_{1}} + \int_{h_{0}}^{h_{1}} G_{t}(s)y(s)ds 
$$ 
with $h_{0},h_{1}$ the points of the biggest sphere with midpoint $t$. In the $1$ dimensional case is either $h_{0}=b_{0}$ or $h_{1}=b_{1}$. Meaning we're still left with 
additive branching recursion. To get rid of this we couple the Russian roulette of these
terms such that there is no branching. <br>

Here is a nasty implementation of this:

In [117]:
from random import random
from math import exp
from numba import njit, prange

@njit(fastmath =True)
def Gb0(t,b0,b1): return (b1-t)/(b1-b0)
@njit(fastmath =True)
def Gb1(t,b0,b1): return (t-b0)/(b1-b0)
@njit(fastmath =True)
def G(t,s,b0,b1): return -(b1-s)*(t-b0)/(b1-b0) if t<s else - (b1-t)*(s-b0)/(b1-b0) 

@njit(fastmath =True)
def Yws(t):
    if t==-1: return exp(-1)
    if t==1: return exp(1)

    if t+1<1-t: 
        h0,h1 = -1,2*t+1
        if random()<0.5:
            return Gb0(t,h0,h1)*Yws(h0)+ 2*Gb1(t,h0,h1)*Yws(h1)
        elif random()<0.5:
            S = h0+random()*(h1-h0)
            return Gb0(t,h0,h1)*Yws(h0)+ 4*G(t,S,h0,h1) * Yws(S)*(h1-h0)
        else:
            return Gb0(t,h0,h1)*Yws(h0)
    else:
        h0,h1 = 2*t-1,1
        if random()<0.5:
            return 2*Gb0(t,h0,h1)*Yws(h0) + Gb1(t,h0,h1)*Yws(h1)
        elif random()<0.5:
            S = h0+random()*(h1-h0)
            return Gb1(t,h0,h1)*Yws(h1) + 4*G(t,S,h0,h1) * Yws(S)*(h1-h0)
        else:
            return Gb1(t,h0,h1)*Yws(h1)

@njit(fastmath=True, parallel=True)
def wssol(t,nsim ):
    total = 0.0
    for _ in prange(nsim):
        total += Yws(t)
    return total/nsim

In [179]:
t,nsim = 0.4,10**5
print((wssol(t,nsim)-exp(t))/exp(t))

0.0016335495054296664


The advantage of Walk on Sphere is that it doesn't need the full Green functions and there is no worrying about meshing.

## Coupled Stratified RMC

$$
\begin{align*}
y(t) &= G_{b_{0}}(t)y_{b_{0}} + G_{b_{1}}(t) y_{b_{1}} + \int_{b_{0}}^{b_{1}} G_{t}(s)f(s)ds \\
 &= G_{b_{0}}(t)y_{b_{0}} + G_{b_{1}}(t) y_{b_{1}} 
 + \int_{b_{0}}^{l} G_{t}(s)f(s)ds + \int_{l}^{b_{1}} G_{t}(s)f(s)ds  .
\end{align*}
$$

We do coupled recursion on those $2$ integral for 
$$
y''=y, y(-1)=e^{-1},y(1)=e.
$$ 
this gives the following nice matrix recursion:

$$
X(t_{1},t_{2})=
\begin{bmatrix} 
G_{b_{0}}(t_{1}) & G_{b_{1}}(t_{1})\\
G_{b_{0}}(t_{2}) & G_{b_{1}}(t_{2})
\end{bmatrix} 
\begin{bmatrix} 
y(b_{0})\\
y(b_{1})
\end{bmatrix} 
+
\begin{bmatrix} 
G_{t_{1}}(U_{1}) & G_{t_{1}}(U_{2}) \\
G_{t_{2}}(U_{1}) & G_{t_{2}}(U_{2}) 
\end{bmatrix}
\begin{bmatrix} 
f_{U_{1}}(U_{1}) & 0 \\
0                & f_{U_{2}}(U_{2}) 
\end{bmatrix}
X(U_{1},U_{2})
$$ 
where $U_{1} = \text{Uniform}(b_{0},l), U_{2} = \text{Uniform}(l,b_{1})$. We implement this with russian roulette:

In [187]:
from random import random
from math import exp
import numpy as np

def Gb0(t,b0,b1): return (b1-t)/(b1-b0)
def Gb1(t,b0,b1): return (t-b0)/(b1-b0)
def G(t,s,b0,b1): return -(b1-s)*(t-b0)/(b1-b0) if t<s else - (b1-t)*(s-b0)/(b1-b0) 

def Xrec(t1,t2):
    l = 2
    if random()*l>1: return np.zeros(2) 
    S1,S2 = random()-1,random()
    X = Xrec(S1,S2)
    x0 = Gb0(t1,-1,1)*exp(-1) + Gb1(t1,-1,1)*exp(1) 
    x0 += G(t1,S1,-1,1)*X[0] + G(t1,S2,-1,1)*X[1]
    x1 = Gb0(t2,-1,1)*exp(-1) + Gb1(t2,-1,1)*exp(1) 
    x1 += G(t2,S1,-1,1)*X[0] + G(t2,S2,-1,1)*X[1]
    return l*np.array([x0,x1])

def solrec(t1,t2,nsim): return sum(Xrec(t1,t2) for _ in range(nsim))/nsim

print(solrec(-0.5,0.3,10**3))
print(exp(-0.5),exp(0.3))


[0.59564983 1.29340874]
0.6065306597126334 1.3498588075760032


## Local RRMC

We demonstrate local RRMC on a grid with $3$ points $[-0.5,0,0.5]$.
The idea is to use local integral representation on overlapping domains
to repeatatly do coupled RMC to estimate the solution on the grid. <br> 

![local RRMC](tests_period_5/tikz%20figures/local%20RRMC/local%20RRMC.png)

We will implement this with recursion in recursion. The outer recursion is a coupled recursion of $3$ points and the inner recursion is just regular RMC where we assume we know the value at adjacent points. This is like walk on rectangle

In [44]:
from random import random
from math import exp
import numpy as np

def Gb0(t,b0,b1): return (b1-t)/(b1-b0)
def Gb1(t,b0,b1): return (t-b0)/(b1-b0)
def G(t,s,b0,b1): return -(b1-s)*(t-b0)/(b1-b0) if t<s else - (b1-t)*(s-b0)/(b1-b0) 

def Y(t,y0,y1,b0,b1): 
    l = 2 # russian roulette rate
    if random()*l>1: return Gb0(t,b0,b1)*y0 + Gb1(t,b0,b1)*y1
    S = b0+random()*(b1-b0)
    return Gb0(t,b0,b1)*y0 + Gb1(t,b0,b1)*y1 + l*G(t,S,b0,b1) * Y(S,y0,y1,b0,b1)*(b1-b0)

def X_slow():
    l = 1.5 # russian roulette rate
    if random()*l>1: return  np.zeros(3)
    X = X_slow()
    x0 = Y(-0.5,exp(-1),X[1],-1,0)
    x1 = Y(0,X[0],X[2],-0.5,0.5)
    x2 = Y(0.5,X[1],exp(1),0,1) 
    return l*np.array([x0,x1,x2]) 

def soltest2(nsim): return sum(X_slow() for _ in range(nsim))/nsim
grid = np.array([-0.5,0,0.5])
nsim = 10**4
sol = np.exp(grid)
ss = soltest2(nsim)
print(f"soltest2= {ss}, %error{(ss-sol)/sol}")

soltest2= [0.62359773 1.00871407 1.66011109], %error[0.02813884 0.00871407 0.00690828]


## Remark
The ideas in the last $3$ algorithms may be combined. It is for example possible to
do WoS inside the local domains of local RRMC.

# Recursive Brownian Motion

## Brownian Motion 

### Definition

### Symmetry

## Recursive Brownian Motion




# TODO
